In [1]:
import json
import os
import string
from stemming.porter2 import stem
import math

In [2]:
data_path = os.path.join(os.getcwd(),'data','Movie_category_indexing.json')
with open(data_path) as f:
    movie_category = json.load(f)

In [3]:
data_path = os.path.join(os.getcwd(),'data','result.json')
with open(data_path) as f:
    data = json.load(f)

In [4]:
#从txt文件中读取并重新构建字典：words_dic
def read_words_indexing():
    data_path = os.path.join(os.getcwd(),'review_indexing.txt')
    file_read = open(data_path,'r',encoding='utf-8')
    
    words_dic={}# key is word, value is {doc_ID,[word_pos1,word_pos2,...]}
    word=''
    for line in file_read.readlines():
        if line[0] != '\t':
            word = line.split(':')[0]
            word_count = line.split(':')[1]
            
        elif line[0] == '\t':
            line = line.strip()
            doc_ID = line.split(" : ")[0]
            word_pos = line.split(" : ")[1].split(",")
            for i in range(0,len(word_pos)):
                word_pos[i] = int(word_pos[i])
            if word not in words_dic.keys():#如果这个单词不在字典里
                words_dic.update({word:{doc_ID : word_pos}})#一二级字典都添加
            elif word in words_dic.keys():#如果这个单词在字典里
                words_dic[word].update({doc_ID : word_pos})#只更新二级字典
                
    return words_dic

words_dic = read_words_indexing()

In [5]:
def calculate_MI(N_11, N_10, N_01, N_00):  # calculate Mutual Information
    N = N_11 + N_10 + N_01 + N_00
    if N_11 != 0:
        term_1 = (N_11 / N) * math.log2((N * N_11) / ((N_10 + N_11) * (N_01 + N_11)))
    else:
        term_1 = 0

    if N_01 != 0:
        term_2 = (N_01 / N) * math.log2((N * N_01) / ((N_00 + N_01) * (N_01 + N_11)))
    else:
        term_2 = 0

    if N_10 != 0:
        term_3 = (N_10 / N) * math.log2((N * N_10) / ((N_10 + N_11) * (N_00 + N_10)))
    else:
        term_3 = 0

    if N_00 != 0:
        term_4 = (N_00 / N) * math.log2((N * N_00) / ((N_00 + N_01) * (N_00 + N_10)))
    else:
        term_3 = 0

    MI = term_1 + term_2 + term_3 + term_4
    return MI

In [6]:
def load_English_stop_words():
    data_path = os.path.join(os.getcwd(),'data','englishST.txt')
    ref_file = open(data_path, "r")
    English_stop_words = []#store English stop words
    for ref_line in ref_file.readlines():
        English_stop_words.append(ref_line.strip())
    return(English_stop_words)

English_stop_words = load_English_stop_words()

In [7]:
def preprocss_review(review_content):
    English_stop_words = load_English_stop_words()
    
    #Delete punctuations
    punctuations = list(string.punctuation)
    punctuations.append('—')
    punctuations.append('\n')
    for p in punctuations:
        review_content = review_content.replace(p, '').strip()
        
    #split review into words
    words = review_content.split(' ')
            
    # To low case
    for i in range(0,len(words)):  
        words[i] = words[i].lower()
            
    #Remove English stop words
    words = [elem for elem in words if elem not in English_stop_words]
#     print(words)
            
    #Stemming
    for i in range(0,len(words)):
        words[i]=stem(words[i].lower())
        
    return words

In [8]:
def get_all_review_code():#return all the exising review code
    all_review_code=[]
    for key_movie,value_movie in data.items():
#         print('Movie Name: ',key_movie)
        for key_year,value_year in value_movie.items():
            for review in value_year[2]:
                review_code=key_movie+'_'+key_year+'_'+review['review_id']
                all_review_code.append(review_code)
    all_review_code = list(set(all_review_code)) #去重（为什么会有重复的项？）
    return all_review_code
            
#get_all_review_code()[0]
len(get_all_review_code())

430283

In [9]:
def takeSecond(elem):#用来排序
    return elem[1]

__光是计算一个review里所有word的MI就要大约45s（当然这和review的长度有关），如果要计算所有review，粗略估计要花费5000小时<br>__
虽然还可以优化，但是我估计如果要提前计算所有的review_MI还是不太现实，建议加一个网页专门用来计算某一篇review的MI（提前告知用户要等待1分钟才能出结果）

In [10]:
%%time
def review_MI(review_code):#review_code的格式是 'movieName_year_reviewID'
    words_MI=[]
    line = review_code
    movie,year,review_id = line.split('_')
#     print(movie,year,review_id)
    
    #Get the category of the movie of this review
    this_review_movie_category = data[movie][year][1].split(',')
#     print('this_review_movie_category = ',this_review_movie_category)
    
    # Get review_content from the dictionary
    review_content = None
    for review in data[movie][year][2]:
        if review['review_id'] == review_id:
            review_content = review["review_detail"]
            
    # Preprocessing the review_content
    words = preprocss_review(review_content)
    words = list(set(words))#去重
    
    # Calculate MI for each word
    for word in words:# For each word
#         print('word =',word)      
#         print(words_dic[word])
        
        N_11,N_10,N_01,N_00 = 0,0,0,0 #paramters of MI
        
        categories=[]# categories of movie reviews that the word appears in
        for review_code in words_dic[word].keys():# For each review that this word appears
            #movie,year = review_code.split('_')[0:2]#存在这种review_code "#cats_the_mewvie_2020_rw5574325",所有这种办法会导致报错
            p=review_code.find("_rw")#找到"_rw"的位置
            movie = review_code[0:p-5]
            year = review_code[p-4:p]
#             print('movie=',movie,', year=',year)
            line = data[movie][year][1]# category line of this movie in the movie dataset
            for category in line.split(','):
                if category not in categories:
                    categories.append(category)
            current_review_category = line.split(',')
#             print(current_review_category)
#             print(set(this_review_movie_category).issubset(current_review_category))
            if set(this_review_movie_category).issubset(current_review_category):
                N_11+=1 #word出现在多少篇属于此类的doc中
            else:
                N_01+=1 #word出现在多少篇不属于此类的doc中
        
#         print('The word appears in categories = ',categories)
        
        
        #找出属于此类的电影：
        same_type_movies_set = {}
        for category in this_review_movie_category:
            if same_type_movies_set == {}:
                same_type_movies_set = set(movie_category[category])
            elif same_type_movies_set != {}:
                same_type_movies_set = same_type_movies_set&set(movie_category[category])
#         print('len(same_type_movies_set) = ',len(same_type_movies_set))
        
        #找出这些电影的影评的review_code
        all_review_codes_belongsToThisCategory=[]#属于此类的doc
        same_type_movies_set = list(same_type_movies_set)
        for movie_code in same_type_movies_set:
            movie,year = movie_code.split("_")
            
            for review in data[movie][year][2]:
                review_id = review['review_id']
                review_code = movie_code+'_'+review_id
                all_review_codes_belongsToThisCategory.append(review_code)
#                 print('review_id=',review_id)
#                 print('review_code=',review_code)
#         print(all_review_codes_belongsToThisCategory)

        #找出所有未出现此word的影评（在属于此类的doc中）
        word_appears_review_codes = list(words_dic[word].keys())#出现过此词的doc
        word_NOT_appears_review_codes = list( set(all_review_codes_belongsToThisCategory) - set(word_appears_review_codes) )
        N_10 = len(word_NOT_appears_review_codes)
#         print('len(all_review_codes_belongsToThisCategory)=',len(all_review_codes_belongsToThisCategory))
#         print('len(word_appears_review_codes)=',len(word_appears_review_codes))

        #在不属于此类的doc中有多少篇未出现此词
        all_review_codes = get_all_review_code()
#         print('len(all_review_codes)=',len(all_review_codes))
        all_review_codes_NOT_belongsToThisCategory = list(set(all_review_codes)-set(all_review_codes_belongsToThisCategory))
#         print('len(all_review_codes_NOT_belongsToThisCategory)=',len(all_review_codes_NOT_belongsToThisCategory))
        N_00 = len(all_review_codes_NOT_belongsToThisCategory) - len(set(all_review_codes_NOT_belongsToThisCategory)&set(word_appears_review_codes))
        
        
#         print('N_11=',N_11,'\nN_10=',N_10,'\nN_01=',N_01,'\nN_00=',N_00)
        word_MI = calculate_MI(N_11, N_10, N_01, N_00)
#         print('word_MI = ',word_MI)
        words_MI.append([word,word_MI])
#         print(words_MI)
#         break
    words_MI.sort(key=takeSecond, reverse=True)#排序
    return words_MI
    

review_MI_example = review_MI("niagara falls_1941_rw2493287")

Wall time: 44.1 s


In [11]:
review_MI_example

[['film', 0.0014616869184980712],
 ['marri', 0.0013581552746259962],
 ['scene', 0.0008401077258319957],
 ['coupl', 0.0006610227002213365],
 ['final', 0.0005490431380410737],
 ['interest', 0.0005222729963865569],
 ['good', 0.00046981182323250267],
 ['make', 0.0004673248449894689],
 ['fall', 0.00046170091623077745],
 ['mind', 0.0004439255002758925],
 ['dont', 0.0004272063762318064],
 ['point', 0.000415115991995731],
 ['back', 0.0004064396351419171],
 ['start', 0.00039713246602490933],
 ['pitt', 0.00038112191382855276],
 ['escap', 0.0003543381715698879],
 ['fact', 0.0003141689870645695],
 ['car', 0.00031332985101126145],
 ['year', 0.00029151627848146046],
 ['attract', 0.00023442590496584544],
 ['attempt', 0.00022586793345104306],
 ['complet', 0.0001970656809437254],
 ['wait', 0.00017860540763841855],
 ['man', 0.0001764442016803109],
 ['absolut', 0.0001713101529091164],
 ['attent', 0.00016163104441688616],
 ['produc', 0.00014640509444661877],
 ['funniest', 0.00013914432609119756],
 ['larg'